In [1]:
%use fuel(2.3.1)

In [2]:
val envs = java.io.File("../../.env")
    .readLines()
    .map {
        it.split("=")[0] to it.split("=")[1].trim('"')
    }.toMap()

In [3]:
val session = envs.get("AOC_SESSION")
val year = 2021
val day = 16

In [4]:
fun getInput(year: Int, day: Int, session: String): String {
    val (_, _, result) = "https://adventofcode.com/$year/day/$day/input"
    .httpGet()
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get().trim()
}

In [5]:
fun submitAnswer(year: Int, day: Int, session: String, level: Int, answer: String): String {
    val (_, _, result) = Fuel
    .post(
        "https://adventofcode.com/$year/day/$day/answer", 
        parameters = listOf("level" to level, "answer" to answer))
    .header("cookie" to "session=$session")
    .responseString()
        
    return result.get()
}

In [6]:
val sample = """D2FE28"""

In [7]:
val input = getInput(year, day, session)

In [8]:
enum class ExpressionType(val value: Int) {
    SUM(0),
    PROD(1),
    MIN(2),
    MAX(3),
    LIT(4),
    GT(5),
    LT(6),
    EQ(7),
    NULL(8);

    companion object {
        fun fromInt(value: Int) = ExpressionType.values().first { it.value == value }
    }
}

In [9]:
class Expression {
    val children = mutableListOf<Expression>()
    var value = 0L
    var version = 0
    var versionSum = 0
    var offset = 0
    var length = 0
    var type = ExpressionType.NULL
    
    constructor(bits: String, offset: Int) {
        this.offset = offset
        var ptr = offset
        version = bits.slice(ptr until ptr + 3).toInt(radix = 2)
        versionSum = version
        type = ExpressionType.fromInt(bits.slice(ptr + 3 until ptr + 6).toInt(radix = 2))
        
        ptr += 6
        if (type == ExpressionType.LIT) {
            while (bits[ptr] != '0') {
                value = value.shl(4)
                value += bits.slice(ptr + 1 until ptr + 5).toInt(radix = 2)
                ptr += 5
            }
            value = value.shl(4)
            value += bits.slice(ptr + 1 until ptr + 5).toInt(radix = 2)
            ptr += 5
        } else {
            if (bits[ptr] == '0') {
                val subLength = bits.slice(ptr + 1 until ptr + 16).toInt(radix = 2)
                ptr += 16
                while (ptr < offset + 22 + subLength) {
                    val child = Expression(bits, ptr)
                    children.add(child)
                    ptr += child.length
                    versionSum += child.versionSum
                }
            } else {
                val subCount = bits.slice(ptr + 1 until ptr + 12).toInt(radix = 2)
                ptr += 12
                repeat(subCount) {
                    val child = Expression(bits, ptr)
                    children.add(child)
                    ptr += child.length
                    versionSum += child.versionSum
                }
            }
            
            assert(children.size >= 1)
            when (type) {
                ExpressionType.SUM -> { value = children.map { it.value }.sum() }
                ExpressionType.PROD -> { value = children.fold(1L, {acc, it -> acc * it.value}) }
                ExpressionType.MIN -> { value = children.map { it.value }.minOrNull()!! }
                ExpressionType.MAX -> { value = children.map { it.value }.maxOrNull()!! }
                ExpressionType.GT -> { value = if (children[0].value > children[1].value) 1 else 0 }
                ExpressionType.LT -> { value = if (children[0].value < children[1].value) 1 else 0 }
                ExpressionType.EQ -> { value = if (children[0].value == children[1].value) 1 else 0 }
                else -> {}
            }
        }
        
        length = ptr - offset
    }
    
    override fun toString(): String {
        return when (type) {
            ExpressionType.NULL -> ""
            ExpressionType.LIT -> value.toString()
            else -> {
                val parts = mutableListOf<String>()
                parts.add("(")
                val op = when (type) {
                    ExpressionType.SUM -> "+"
                    ExpressionType.PROD -> "*"
                    ExpressionType.MIN -> "min"
                    ExpressionType.MAX -> "max"
                    ExpressionType.GT -> ">"
                    ExpressionType.LT -> "<"
                    ExpressionType.EQ -> "="
                    else -> "" // should not enter here
                }
                parts.add(op)
                parts.addAll(children.map { it.toString() })
                parts.add(")")
                parts.joinToString(" ")
            }
        }
    }
    
    companion object {
        fun fromHex(hex: String) = Expression(
            hex.map { it.toString().toInt(radix = 16).toString(radix = 2).padStart(4, '0') }.joinToString(""),
            0
        )
    }
}

In [10]:
val samples = listOf(
    sample,
    "38006F45291200",
    "EE00D40C823060",
    "8A004A801A8002F478",
    "620080001611562C8802118E34",
    "C0015000016115A2E0802F182340",
    "A0016C880162017C3686B18A3D4780",
    "C200B40A82",
    "04005AC33890",
    "880086C3E88112",
    "CE00C43D881120",
    "D8005AC2A8F0",
    "F600BC2D8F",
    "9C005AC2F8F0",
    "9C0141080250320F1802104A08",
    input,
)

In [11]:
samples.forEach { 
    println(Expression.fromHex(it))
}

2021
( < 10 20 )
( max 1 2 3 )
( min ( min ( min 15 ) ) )
( + ( + 10 11 ) ( + 12 13 ) )
( + ( + 10 11 ) ( + 12 13 ) )
( + ( + ( + 6 6 12 15 15 ) ) )
( + 1 2 )
( * 6 9 )
( min 7 8 9 )
( max 7 8 9 )
( < 5 15 )
( > 5 15 )
( = 5 15 )
( = ( + 1 3 ) ( * 2 2 ) )
( + ( * 445572 ( = ( + 8 8 11 ) ( + 2 12 7 ) ) ) ( * 238471898 ( < 5 125441674 ) ) ( * 17226 ( > 265949656 90 ) ) ( min 92 138 189 ) ( max 333045 1 635818 3837 ) ( * ( > 32733 32733 ) 10192146 ) ( min ( * ( min ( min ( max ( + ( + ( min ( max ( * ( * ( max ( max ( max ( + ( * ( min ( * ( * ( max 2478414062 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ( * 215 ) 12 ( * 863908 ( > ( + 13 12 3 ) ( + 6 13 12 ) ) ) ( + 752712 4152413765 ) ( * 195 ( < ( + 12 3 14 ) ( + 10 13 13 ) ) ) ( * 130 ( < 2188 2188 ) ) ( * ( > 528325 1666 ) 3205 ) 146272 ( + ( * 3 7 8 ) ( * 11 3 6 ) ( * 6 5 2 ) ) ( * 220 50 62 54 135 ) ( * ( > ( + 10 10 8 ) ( + 13 11 14 ) ) 147 ) ( * ( = 45179 45179 ) 17086 ) ( * 5802 ( < 595085 595085 ) ) 3007 ( * 4419 ( < 8 16198656 ) ) 

In [12]:
fun partOne(input: String): String {
    return Expression.fromHex(input).versionSum.toString()
}

In [13]:
partOne(sample)

6

In [14]:
val partOneAns = partOne(input)
partOneAns

871

In [ ]:
submitAnswer(year, day, session, 1, partOneAns)

In [15]:
fun partTwo(input: String): String {
    return Expression.fromHex(input).value.toString()
}

In [16]:
partTwo(sample)

2021

In [17]:
val partTwoAns = partTwo(input)
partTwoAns

68703010504

In [ ]:
submitAnswer(year, day, session, 2, partTwoAns)